This is the second part of [iSynthesis](https://github.com/cimm-kzn/iSynthesis) tutorial

# Prepare reaction synthetic rules
## 1. Read reaction data

Reactions should be presented in RDF format. It is possible to create one-component (single) and two-component (double) reaction rules with our algorithm.

In [1]:
from CGRtools import RDFRead, ReactionContainer
from pickle import dump

In [2]:
reactions = RDFRead('standardized_golden_dataset.rdf').read()
len(reactions)

1851

## 2. Standardize reactions and create reaction rules

In [3]:
def create_rule(reaction, center, environment  = None, hybridization= 'standard', neighbors='standard'):
    reactants = reaction.reactants
    products = reaction.products
    new_cgr = ~reaction
    c_a = new_cgr.center_atoms    # c_a is a range of reaction center atoms.
    clevege = set(reaction.reactants).difference(reaction.products)    # clevege a leaving group atoms.
    coming = set(reaction.products).difference(reaction.reactants)     # coming is incoming group atoms.   
    set_at_env = set(center).difference(clevege.union(coming).union(c_a)) # set_at_env is environment atoms excluding leaving and incoming groups atoms.

    flag = 0   # flag is a count of molecules which have atoms of reaction center.
    for x in reaction.reactants:
        if set(c_a).intersection(x):
            flag += 1

    if environment:
        re_1 = []
        for molecule in reactants:
            if set(center).intersection(molecule):
                # aug_env is range maping of atoms which contain in environment of reaction.
                aug_env = molecule.augmented_substructure(set(c_a).intersection(molecule), deep=environment)
                # re_1 is a range of reactants with rule atoms and atoms from a certain depth.
                re_1.append(molecule.substructure((set(center).intersection(molecule).union(aug_env)), as_query=True))
        pr_1 = []
        for molecule in products:
            if set(center).intersection(molecule):
                # aug_env is range maping of atoms which contain in environment of reaction.
                aug_env = molecule.augmented_substructure(set(c_a).intersection(molecule), deep=environment)
                # pr_1 is a products with rule atoms and atoms from a certain depth.
                pr_1.append(molecule.substructure((set(center).intersection(molecule).union(aug_env)), as_query=True))
    else:
        # re_1 is a range of reactants with rule atoms.
        re_1 = [molecule.substructure(set(center).intersection(molecule), as_query=True) for molecule in reactants if
                set(center).intersection(molecule)]
        # pr_1 is a products with rule atoms.
        pr_1 = [molecule.substructure(set(center).intersection(molecule), as_query=True) for molecule in products if
                set(center).intersection(molecule)]

    Out_re = ReactionContainer(reactants=re_1, products=pr_1)     # Out_re is a reaction rule
    Out_re.meta['composition'] = flag

    for molecule in Out_re.molecules():     # Removing information of ring sizes and count of hydrogens.
        molecule._rings_sizes = {x: () for x in molecule._rings_sizes}
        molecule._hydrogens = {x: () for x in molecule._hydrogens}

    if hybridization == 'all':     # hybridization == 'all' is labels hybridization on all atoms.
        pass

    elif hybridization == 'env' and environment:     # hybridization == 'env' is removing labels atom hybridization on central atoms.

        only_c_a = set(center).intersection(c_a)     # only_c_a is a range of atoms environment atoms.
        for molecule in Out_re.reactants:
            if set(molecule).intersection(only_c_a):
                for del_hyb_atom in set(molecule).intersection(only_c_a):
                    molecule._hybridizations[del_hyb_atom] = ()

        for molecule in Out_re.products:
            if set(molecule).intersection(only_c_a):
                for del_hyb_atom in set(molecule).intersection(only_c_a):
                    molecule._hybridizations[del_hyb_atom] = ()

    elif hybridization == None:     # hybridization == None is removing lables hybridization in all atoms.
        for molecule in Out_re.reactants:
            for del_hyb_atom in set(molecule):
                molecule._hybridizations[del_hyb_atom] = ()
        for molecule in Out_re.products:
            for del_hyb_atom in set(molecule):
                molecule._hybridizations[del_hyb_atom] = ()

    elif hybridization == 'standard':     # hybridization on all atoms except outgoing and incoming groups 'standard'

        for del_hyb_atom in clevege:         # removing hybridization on del_hyb_atom in reactant molecules then product molecules.
            for molecule in Out_re.reactants:
                if del_hyb_atom in molecule:
                    molecule._hybridizations[del_hyb_atom] = ()

        for del_hyb_atom in coming:         # removing hybridization on del_hyb_atom in products molecules then product molecules.
            for molecule in Out_re.products:
                if del_hyb_atom in molecule:
                    molecule._hybridizations[del_hyb_atom] = ()

    if neighbors == 'all':     # neighbors == 'all' is labels neighbors  on all atoms.
        pass

    elif neighbors == 'standard':     # neighbors == 'standard'. Removing label atom neighbors on environment atoms excluding leaving and incoming groups atoms
        for at_env in set_at_env:
            for molecule in Out_re.reactants:
                if at_env in molecule.atoms_numbers:
                    molecule._neighbors[at_env] = ()
            for molecule in Out_re.products:
                if at_env in molecule.atoms_numbers:
                    molecule._neighbors[at_env] = ()

    elif neighbors == 'f_e' and environment:     # neighbors == 'f_e'. Removing label atom neighbors on central atoms omly.
        for molecule in Out_re.reactants:
            f_e = set(molecule.augmented_substructure(set(c_a).intersection(molecule), deep=1)).difference(c_a)
            for atom in molecule:
                if atom not in f_e:
                    molecule._neighbors[atom] = ()
        for molecule in Out_re.products:
            f_e = set(molecule.augmented_substructure(set(c_a).intersection(molecule), deep=1)).difference(c_a)
            for atom in molecule:
                if atom not in f_e:
                    molecule._neighbors[atom] = ()
   
    elif neighbors == 'r_c':  # neighbors == 'r_c'. Removing label atom neighbors on all atoms excluding reaction center atoms.
        for molecule in Out_re.reactants:
            for atom in molecule:
                if atom not in c_a:
                    molecule._neighbors[atom] = ()
        for molecule in Out_re.products:
            for atom in molecule:
                if atom not in c_a:
                    molecule._neighbors[atom] = ()

    elif neighbors == 'env':     # neighbors == 'env'. Removing label atom neighbors on central atoms.
        for molecule in Out_re.reactants:
            for atom in molecule:
                if atom in c_a:
                    molecule._neighbors[atom] = ()

        for molecule in Out_re.products:
            for atom in molecule:
                if atom in c_a:
                    molecule._neighbors[atom] = ()

    Out_re.flush_cache()     # flush_cache() need to label changes have been applied. 
    return Out_re

In [4]:
reaction_rules =[]
for reaction in reactions:
    try:
        reaction.canonicalize()  # standardization procedure
        cgrs = ~reaction
    except ValueError: 
        raise reaction

    #skip reaction if there is a radical atom
    if cgrs.center_atoms: 
        if any(x.is_radical or x.p_is_radical for _, x in cgrs.atoms()):
            raise reaction

    for extend_center in reaction.extended_centers_list:
        reaction_rules.append(create_rule(reaction, extend_center))
len(reaction_rules)

2046

#### Check out how rule corresponds to reaction:

In [5]:
reactions[1]

In [6]:
reaction_rules[1]


## 3. Separate rules by number of reactants

In [7]:
single_rules_path = '../data/rules/single_rules.pickle'
double_rules_path = '../data/rules/double_rules.pickle'
indexed_single_rules_path = '../data/rules/single.pickle'
indexed_double_rules_path = '../data/rules/double.pickle'
functional_groups_path = '../data/rules/groups.pickle'

In [8]:
single, double, multi = [], [], []

for rule in reaction_rules:
    c = rule.meta['composition']
    if c == 1:
        single.append(rule)
    elif c == 2:
        double.append(rule)
    else:
        multi.append(rule)

print(f'number of one-component extracted rules: {len(single)}')
print(f'number of two-component extracted rules: {len(double)}')
print(f'number of multi-component extracted rules: {len(multi)}')

number of one-component extracted rules: 459
number of two-component extracted rules: 1265
number of multi-component extracted rules: 322


### Save reaction rules in binary pickled files:

In [9]:
with open(single_rules_path, 'wb') as e:
    dump(single, e)

with open(double_rules_path, 'wb') as e:
    dump(double, e)

## 4. Create common functional groups dictionary from all rules and index rules:


In [10]:
group_dict = {}

def create_indexed_rules_dict(rules, group_dict):
    indexed_rules = {}
    for n, rule in enumerate(rules, start=1):
        for group in rule.reactants:
            i = len(group_dict) + 1
            g = group.copy()
            g.remap(dict(zip(g.atoms_numbers, [x + 1 for x in range(len(g))])), copy=True)
            ii = group_dict.setdefault(bytes(g), (i, g))
            indexed_rules.setdefault(rule, []).append(ii[0])
    return indexed_rules, group_dict

single_with_groups, group_dict = create_indexed_rules_dict(single, group_dict)
double_with_groups, final_group_dict = create_indexed_rules_dict(double, group_dict)
print(f'Number of functional groups: {len(final_group_dict)}')

Number of functional groups: 1145


## 5. Save indexed rules and extracted functional groups into binary files:

In [11]:
with open(functional_groups_path, 'wb') as f:
    dump({i[0]: i[1] for _, i in final_group_dict.items()}, f)

with open(indexed_single_rules_path, 'wb') as f:
    dump(single_with_groups, f)

with open(indexed_double_rules_path, 'wb') as f:
    dump(double_with_groups, f)